# RunPod GPU 관리 노트북

REST API 직접 호출 방식 (serverless-mlops 패턴).
GPUType enum으로 타입 안전하게 GPU를 지정하고, pandas DataFrame으로 Pod 목록을 확인한다.

| GPU       | VRAM | ~$/hr | 용도          |
|-----------|------|------|-------------|
| RTX 4090  | 24GB | 0.39 | 2B 모델 학습/추론 |
| L40S      | 48GB | 0.74 | 7B 모델 서빙    |
| A100 80GB | 80GB | 1.64 | 대형 모델       |

In [ ]:
import os, sys
from dotenv import load_dotenv

load_dotenv()
sys.path.insert(0, os.path.abspath(".."))

from util.runpod_client import *

print("RunPod REST 클라이언트 로드 완료")

In [ ]:
# 셀 2: Pod 목록 조회 → DataFrame
import pandas as pd

pod_list = pods()
if pod_list:
    df = pd.DataFrame(pod_list)
    display(df)
else:
    print("실행 중인 Pod이 없습니다.")

In [ ]:
# 셀 3: Pod 생성 (GPUType enum 사용)
pod_id = create(
    name="my-workspace",
    gpu_id=GPUType.NVIDIA_GEFORCE_RTX_4090,
    gpu_count=1,
    volume=50,
    image_name="runpod/pytorch:2.4.0-py3.11-cuda12.4.1-devel-ubuntu22.04",
)
print(f"생성된 Pod ID: {pod_id}")

In [ ]:
# 셀 4: 특정 Pod 상태 조회
# pod_id = "POD_ID"
pod_info = pod(pod_id)
pod_info

In [ ]:
# 셀 5: Pod 삭제
# pod_id = "POD_ID"
result = delete("6bo9b41ss21l6t")
print(f"삭제 결과: {result}")

In [ ]:
# 셀 6: VLM 서빙 Pod 생성
vlm_pod_id = create_vlm_pod(
    name="qwen3-vl-4b-serving",
    model_name="Qwen/Qwen3-VL-4B-Instruct",
    gpu_id=GPUType.NVIDIA_GEFORCE_RTX_5080,
    max_model_len=4096,
)
print(f"VLM Pod ID: {vlm_pod_id}")

In [ ]:
# 셀 7: vLLM 연결 테스트
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

VLLM_BASE_URL = "https://api.runpod.ai/v2/i1sc2eu2b68yuj/openai/v1" # 예: http://your-pod-ip:8000/v1
MODEL_NAME = "Qwen/Qwen3-VL-4B-Instruct" 

if VLLM_BASE_URL != "여기에_URL을_입력하세요":
    llm = ChatOpenAI(
        base_url=VLLM_BASE_URL,
        model=MODEL_NAME,
        temperature=0,
        api_key=os.getenv("RUNPOD_API_KEY")
    )

    response = llm.invoke([
        HumanMessage(content="안녕! 너는 어떤 모델이니? 도구 호출이 가능하니?")
    ])
    print("\n[응답 결과]")
    print(response.content)
else:
    print("VLLM_BASE_URL을 입력해 주세요.")